In [20]:
import spacy
import os
import nltk
from current_project_programs.preprocess import Preprocess
from nltk.parse.stanford import StanfordParser
from nltk.corpus import stopwords
from spacy import displacy
from spacy.matcher import Matcher
import json
from spacy.tokens import Doc, Span, Token
#import neuralcoref
import time
from nltk.corpus import wordnet
import numpy as np

def load_text_files(filename, dir='dataset/organizations/'):
    list_of_directory = os.listdir(dir)
    dataset = {}
    test_sentences = []
    for file in list_of_directory:
        if (not filename and file.endswith('.txt')) or file.endswith(filename):
            current = open(dir+file,"r",encoding='ISO-8859-1')
            list_of_paragraphs = []
            for count, line in enumerate(current):
                list_of_paragraphs.append(line.strip().replace('ï»¿','').replace('â€“','-'))
            dataset[file] = list_of_paragraphs
       #dataset['Praveen.txt'] = test_sentences
    return dataset

def feature_extraction(word):
    definition,synonyms,antonyms,hyper,hypo,mero,holo = [],[],[],[],[],[],[]
    for syn in wordnet.synsets(word):
        definition.append(syn.definition())
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
        for hypernym in syn.hypernyms()[:2]:
            hyper.append(hypernym.lemma_names())
        for hyponym in syn.hyponyms()[:2]:
            hypo.append(hyponym.lemma_names())
        for meronym in syn.part_meronyms()[:2]:
            mero.append(meronym.lemma_names())
        for holonym in syn.part_holonyms()[:2]:
            holo.append(holonym.lemma_names())
    return definition,synonyms,antonyms,hyper,hypo,mero,holo

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    if doc[start].ent_iob_=='B' and (len(doc)==end or doc[end].ent_type_!='GPE'):
        sent = span.sent  # Sentence containing matched span
        extraction = {"template":"PART", "sentences":[sent.text]}
        arguments = {}
        # Append mock entity for match in displaCy style to matched_sents
        # get the match span by ofsetting the start and end of the span with the
        # start and end of the sentence in the doc
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            "label": "PLACE_IN_PLACE",
        }]
        matched_sents.append({"text": sent.text, "ents": match_ents})
        arguments["1"], arguments["2"] = ''.join(str(span)).split(',')
        extraction["arguments"] = arguments
        result["extraction"].append(extraction)
       
def show_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{
        "start": span.start_char - sent.start_char,
        "end": span.end_char - sent.start_char,
        "label": "PLACE",
    }]
    matched_sents.append({"text": sent.text, "ents": match_ents})
   
def merge_ents(doc):
    #print_token_attr(doc)
    with doc.retokenize() as retokenizer:
        for ent in doc.ents:
            retokenizer.merge(ent)
           
    i = 0
    while i<len(doc):
        token = doc[i]
        if token.pos_=='PROPN':
            if token.dep_=='compound':
                tail = token
                while tail.i+1<len(doc) and tail.nbor().dep_=='compound':
                    tail = tail.nbor()
                if tail.i>token.i:
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[token.i:tail.i+1])
        i += 1
       
    i = 0    
    while i<len(doc):
        token = doc[i]
        if token.pos_=='PROPN':
            if token.dep_ == 'nmod':
                if token.head.lemma_.lower() in place_lemmas:
                    place_type = token.head
                    arr = [x.text.strip()+' '+place_type.lemma_ for x in doc[token.i:place_type.i+1] if x.pos_=='PROPN' and x.lemma_.lower() not in place_lemmas]
                    if not (place_type.i+1<len(doc) and place_type.nbor().lemma_.lower()=='of'):
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[token.i:place_type.i+1], attrs={"ENT_TYPE":'GPE'})
                        if len(arr)>1:
                            doc[token.i]._.grouping_type = "SIBLING"
                            doc[token.i]._.list = arr
        i += 1
    return doc

def merge_siblings(doc):
    #print_token_attr(doc)
    i = 0
    while i<len(doc):
        token = doc[i]
        if token.pos_=='CCONJ':
            conj = token.text
            a = token.head.i
            b = token.i
            while b<len(doc) and doc[b].text != [x for x in token.head.children][-1].text:
                b += 1                
            cur = token.head
            ent_type = {}
            ent_type[doc[b].ent_type_] = 1
            if cur.ent_type_ not in ent_type:
                ent_type[cur.ent_type_] = 1
            else:
                ent_type[cur.ent_type_] += 1                
            while cur.head.pos_=='PROPN':
                cur = cur.head
                if cur.ent_type_ not in ent_type:
                    ent_type[cur.ent_type_] = 1
                else:
                    ent_type[cur.ent_type_] += 1
                a = cur.i
                if cur.head == cur:
                    break            
            mod = ''
            attr = {'ENT_TYPE': None}
            if doc[a].dep_=='nmod':
                mod = doc[a].head.lemma_
                if mod.lower() in place_lemmas:
                    attr = {"ENT_TYPE": 'GPE'}
            else:
                attr = {"ENT_TYPE": [k for k, v in sorted(ent_type.items(), key=lambda item: item[1])][-1]}
            if attr["ENT_TYPE"] == 'GPE':
                with doc.retokenize() as retokenizer:
                    retokenizer.merge(doc[a:b+1], attrs=attr)
                doc[a]._.grouping_type = "SIBLING"
                doc[a]._.list = [y.strip()+' '+mod for x in doc[a].text.split(conj) for y in x.split(',')]
                i = a
                #print(doc[a:b+1], attrs)
        i += 1
    return doc

def merge_ancestors(doc):
    #print_token_attr(doc)
    i = 0
    while i<len(doc):
        token  = doc[i]
        if token.ent_type_=='GPE':
            if token.dep_ == 'poss':
                if token.head.ent_type_=='GPE':
                    parent = token.text.strip()
                    child = token.head.text.strip()
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[token.i:token.head.i+1], attrs={"ENT_TYPE":'GPE'})
                    doc[i]._.grouping_type = "CHILD_PARENT"
                    doc[i]._.list = [child, parent]
            if i+1<len(doc) and doc[i+1].text==',':
                if token.head.dep_=='prep':
                    tail = token
                    while tail.i+2<len(doc) and doc[tail.i+1].text==',' and doc[tail.i+2].ent_type_=='GPE':
                        tail = doc[tail.i+2]
                    if tail!=token:
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[i:tail.i+1], attrs={"ENT_TYPE":'GPE'})
                        doc[i]._.grouping_type = "CHILD_PARENT"
                        doc[i]._.list = [y.strip() for y in doc[i].text.split(',')]
                else:
                    result = [token]
                    j = 0
                    while j<len(result):
                        result.extend([x for x in result[j].children if x.ent_type_=='GPE'])
                        j+=1
                    if len(result)>1:
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[i:result[-1].i+1], attrs={"ENT_TYPE":'GPE'})
                        doc[i]._.grouping_type = "CHILD_PARENT"
                        doc[i]._.list = [y.strip() for y in doc[i].text.split(',')]
        elif token.pos_=='NOUN' and token.lemma_.lower() in place_lemmas:
            preps = [child for child in token.children if child.dep_ == 'prep' and child.lemma_.lower() in ['of']]
            for prep in preps:
                children = [x for x in prep.rights if x.pos_=='PROPN']
                if children:
                    parents = [x for x in token.lefts if x.pos_ in ['DET','PROPN','ADJ']]
                    parent = [x for x in parents if x.pos_=='PROPN']
                    if parent:
                        ls = [children[-1].text, parent[0].text]
                        with doc.retokenize() as retokenizer:
                            retokenizer.merge(doc[parents[0].i:children[-1].i+1], attrs={"ENT_TYPE":'GPE','POS':'PROPN'})
                        doc[parents[0].i]._.grouping_type = "CHILD_PARENT"
                        doc[parents[0].i]._.list = ls
                        i = parents[0].i
                    else:
                        parent = [x for x in parents if x.ent_type_=='NORP' and x.head==token]
                        gpe_text = get_gpe_from_norp(parent[0].lemma_.lower()) if parent else None
                        if gpe_text:
                            ls = [children[-1].text, gpe_text]
                            with doc.retokenize() as retokenizer:
                                retokenizer.merge(doc[parents[0].i:children[-1].i+1], attrs={"ENT_TYPE":'GPE','POS':'PROPN'})
                            doc[parents[0].i]._.grouping_type = "CHILD_PARENT"
                            doc[parents[0].i]._.list = ls
                            i = parents[0].i
        i += 1
    return doc

def print_token_attr(doc):
    for token in doc:
        #print(token.text, token.ent_type_, token.ent_iob_, token.lemma_, token.pos_, token.tag_, token.shape_, token.is_alpha, token.is_stop, token.left_edge, token.right_edge, token.ent_kb_id_, token.norm_, token.subtree, token.n_rights, token.conjuncts, token.ancestors, token.head, token.dep_, [x.text for x in token.children])
        #print(token.text, token.lemma_, token.ent_type_, token.pos_, token.left_edge, token.right_edge, token.ent_kb_id_,
        #      token.norm_, [x for x in token.subtree], token.n_rights, token.conjuncts, [x for x in token.ancestors],
        #      token.head, token.dep_, [x.text for x in token.children])
        print(token.text, token.ent_type_, token._.list if token._.grouping_type in ['CHILD_PARENT', 'SIBLING'] else '',
              token.pos_, token.dep_, token.head,[x for x in token.lefts], [x for x in token.rights])

def insert_part_instance(doc, place, in_place):
    print(place, 'in', in_place)
    extraction = {"template":"PART", "sentences":[doc[:].sent.text]}
    arguments = {}
    arguments["1"], arguments["2"] = place, in_place
    extraction["arguments"] = arguments
    result["extraction"].append(extraction)
   
def insert_buy_instance(doc, buyer, item, price, quantity, source):
    print(buyer, 'bought', quantity, item, 'for', price, 'from', source)
    extraction = {"template":"BUY", "sentences":[doc[:].sent.text]}
    arguments = {}
    arguments["1"], arguments["2"], arguments["3"], arguments["4"], arguments["5"] = buyer, item, price, quantity, source
    extraction["arguments"] = arguments
    result["extraction"].append(extraction)
   
def insert_ancestor_list(doc, arr):
    if len(arr)>1:
        for x in range(len(arr)-1):
            for y in range(x+1,len(arr)):
                insert_part_instance(doc, arr[x], arr[y])
               
def insert_child_of_list(doc, child, arr):
    for parent in arr:
        insert_part_instance(doc, child, parent)
       
def insert_on_type(doc, child, parent):
    if child._.grouping_type in ['CHILD_PARENT', 'SIBLING']:
        for child_place in child._.list:
            if parent._.grouping_type in ['CHILD_PARENT', 'SIBLING']:
                insert_child_of_list(doc, child_place, parent._.list)
            else:
                insert_part_instance(doc, child_place, parent.text)
    else:
        if parent._.grouping_type in ['CHILD_PARENT', 'SIBLING']:
            insert_child_of_list(doc, child.text, parent._.list)
        else:
            insert_part_instance(doc, child.text, parent.text)
           
def get_gpe_from_norp(word):
    for defn in feature_extraction(word)[0]:
        if defn.startswith('of or relating to'):
            for ent in nlp_wo_pipe(defn).ents:
                if ent.label_=='GPE':
                    return ent.text
    return None

def extract_place_in_place(doc):
    i = 0    
    #print_token_attr(doc)
    while i<len(doc):
        token = doc[i]
        if token.pos_ == 'PROPN':
            if token.ent_type_=='GPE':
                if token._.grouping_type == 'CHILD_PARENT':
                    insert_ancestor_list(doc, token._.list)
            if token.dep_=='nsubj':
                if token.head.lemma_.lower()=='be':
                    nouns = [child for child in token.head.children if child.pos_=='NOUN' and child.lemma_.lower() in place_lemmas]
                    preps = [child for noun in nouns for child in noun.children if child.dep_ == 'prep' and child.lemma_.lower() in ['of','in']]
                    for prep in preps:
                        for child in prep.children:
                            if child.ent_type_=='GPE':
                                insert_on_type(doc, token, child)                  
                            elif child.pos_ == 'PROPN':
                                insert_part_instance(doc, token.text, child.text)
                            elif child.pos_=='NOUN' and child.lemma_.lower() in place_lemmas:
                                for c in child.children:
                                    if c.pos_=='PROPN':
                                        insert_on_type(doc, token, c)
            elif token.dep_=='poss':
                if token.ent_type_=='GPE':
                    if token.head.pos_ == 'PROPN':
                        insert_on_type(doc, token.head, token)
            else:
                pass
        elif token.pos_=='ADP':
            if token.dep_=='prep':
                if token.lemma_.lower()=='in':
                    child = token.head
                    if child.ent_type_ in ['GPE','LOC','LOC DET']:
                        parents = [x for x in token.rights if x.ent_type_=='GPE']
                        if parents:
                            for parent in parents:
                                insert_on_type(doc, child, parent)
                        else:
                            if token.i+1<len(doc) and token.nbor().ent_type_=='GPE':
                                parent = token.nbor()
                                insert_on_type(doc, child, parent)
                    elif child.lemma_.lower() in place_lemmas:
                        child = None
        i += 1
    return doc

#Praveen
def extract_all_named_entities(doc):
    template = {"buyer":"","item":"","price":"","quantity":"","source":""}
    entities = [ent for ent in doc.ents if ent.label_ in ["BUY","MONEY","ORG","PERSON","PRODUCT","PERCENT","CARDINAL"]]
    print("Entities",entities)
    list_of_templates = []
   
    #print("Code in first rule: (X acquired/purchased Y for dollars)")
    #print("**************************************")
    def constraint_check(entities):
        list_t = []
        if len(entities)==3:
            template = {"buyer":"","item":"","price":"","quantity":"","source":""}
            for ent in ["acquire","purchase","buy","brought","purchased","acquired"]:
                    if ent.strip()==entities[1].text:
                        template["buyer"]= entities[0].text
                        template["item"]= entities[2].text
            for ent in ["acquired by","purchased by"]:
                    if ent.strip()==entities[1].text:
                        template["buyer"]= entities[2].text
                        template["item"] = entities[0].text
            if len(template["buyer"])>0 and len(template["item"])>0:
                list_t.append(template)
        else:
            for index in range(len(entities)):
                template = {"buyer":"","item":"","price":"","quantity":"","source":""}
                for ent in ["acquire","purchase","buy","brought","purchased","acquired","acquisition"]:
                    if (entities[index].text.strip()==ent.strip()):
                        if index+2<len(entities):
                            if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"] and entities[index+2].label_!="MONEY":
                                template["buyer"]= entities[index-1].text
                                template["item"] = entities[index+1].text
                        else:
                            if index-1>0 and index+1<len(entities):
                                if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"]:
                                    template["buyer"]= entities[index-1].text
                                    template["item"] = entities[index+1].text
                           
                if len(template["buyer"])>0 and len(template["item"])>0:
                    list_t.append(template)  
                for ent in ["acquired by","purchased by"]:
                    if entities[index].text.strip()==ent and index-1 and index+1:
                        if index+2<len(entities):
                            if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"] and entities[index+2].label_!="MONEY":
                                template["buyer"]= entities[index-1].text
                                template["item"] = entities[index+1].text
                        else:
                            if entities[index-1].label_  in ["ORG","PERSON","PRODUCT"] and entities[index+1].label_ in ["ORG","PERSON","PRODUCT"]:
                                template["buyer"]= entities[index-1].text
                                template["item"] = entities[index+1].text
                if len(template["buyer"])>0 and len(template["item"])>0:
                    list_t.append(template)
        return list_t

    if len(entities)>=3:
        list_of_templates = constraint_check(entities)
    if len(template["buyer"])==0:
        for ent in entities:      
                head = ent.root.head
                for token in head.children:
                    if (token.pos_ in ["PROPN","NOUN"] and token.dep_=="nsubj" and token.ent_type_ in ["ORG","PERSON","PRODUCT"] and token.head.pos_=="VERB"):
                        template["buyer"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="nsubj" and token.head.pos_=="VERB" and token.head.dep_=="ROOT"):
                        template["buyer"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="appos" and token.head.pos_=="NOUN" and token.head.dep_!="dobj"):
                        template["buyer"] = token.text
                    if (token.pos_=="VERB" and token.dep_=="relcl" and token.head.pos_=="PROPN"):
                        template["buyer"] = token.head.text
                    if (token.pos_ in ["PROPN","NOUN"] and  token.dep_ =="dobj" and token.head.pos_=="VERB" and token.head.ent_type_=="BUY"):
                        template["item"] = token.text
                    if (len(template["item"])!=0 and token.pos_ in ["PROPN","NOUN","ADV"] and token.dep_ in ["dobj","pobj","nsubjpass"] and token.head.pos_=="VERB" and token.ent_type_=="BUY" and token.head.dep_!="advcl"):
                        template["item"] = token.text
                    if (len(template["item"])!=0 and token.pos_=="PROPN" and token.dep_=="dobj" and token.head.ent_type_=="BUY" and token.ent_type_ in ["ORG","PERSON","PRODUCT"]):
                        template["item"] = token.text
                    if (len(template["item"])!=0 and token.pos_=="PROPN" and token.dep_=="appos" and token.head.pos_=="NOUN" and token.head.head.ent_type_=="BUY"):
                        template["item"] = token.text
                    if (token.pos_=="PROPN" and token.dep_=="compound" and token.head.pos_=="PROPN"):
                        template["item"] = token.text
                    if (token.pos_=="NUM" and token.dep_ in ["npadvmod","pobj","nummod"] and token.ent_type_ in ["MONEY","CARDINAL"]):
                        if token.head.pos_=="NOUN":
                            template["price"] = str(token.text)+" "+str(token.head.text)
                        else:
                            template["price"] = token.text
                    if (token.pos_ in ["PROPN","ADP"] and token.dep_=="pobj" and  token.head.pos_ in ["ADP","SCONJ"] and token.head.dep_=="prep" and token.head.head.pos_ in ["VERB"] and token.ent_type_ in ["ORG","PERSON","PRODUCT","GPE"]):
                         template["source"] = token.text
                    if (token.ent_type_=="PERCENT" and token.pos_=="NOUN" and token.dep_ in ["compound","dobj","npadvmod","nmod"]):
                        template["quantity"] = token.text
                    #print(template)
        if len(template["buyer"])>0 and len(template["item"])>0:
            list_of_templates.append(template)
        #print("first_rule",template)  
        template = {"buyer":"","item":"","price":"","quantity":"","source":""}
        #print("Code in second rule:(X  was acquired/purchased Y for dollars)")
        #print("**************************************")
        for ent in entities:
                head = ent.root.head    
                for token in head.children:
                    if (token.pos_=="VERB" and token.dep_=="acl" and token.ent_type_ in ["BUY"]):
                        template["item"]=head
                    if (token.pos_=="PROPN" and token.dep_=="nsubjpass" and  token.ent_type_ in ["ORG","PERSON","PRODUCT"]):
                            template["item"] = token
                    if (token.pos_=="PROPN" and token.dep_=="pobj") and (token.ent_type_ in ["ORG","PERSON","PRODUCT"]):
                            template["buyer"] = token
                    if (token.ent_type_=="MONEY" and token.pos_=="NUM" and token.dep_ in ["npadvmod","pobj","nummod"]):
                            template["price"] = token
                    if (token.ent_type_=="PERCENT" and token.pos_=="NOUN" and token.dep_ in ["compound","dobj","npadvmod"]):
                        template["quantity"] = token
        if len(template["buyer"])>0 and len(template["item"])>0:
            list_of_templates.append(template)
            template = {"buyer":"","item":"","price":"","quantity":"","source":""}
    #print("second_rule",template)
    list_of_templates= [dict(y) for y in set(tuple(x.items()) for x in list_of_templates)]
   
    print("Final Template:",list_of_templates)
    for template in list_of_templates:
        insert_buy_instance(doc, template['buyer'] if len(template["buyer"])>0 else template['buyer'],
                            template['item'] if len(template["item"])>0 else template['item'],
                            template['price'] if len(template["price"])>0 else template['price'],
                            template['quantity'] if len(template["quantity"])>0 else template['quantity'],
                            template['source'] if len(template["source"])>0 else template['source'])
    return list_of_templates
#Praveen

if __name__=='__main__':
    s = time.time()
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nlp = spacy.load('en_core_web_sm')
    nlp_wo_pipe = spacy.load('en_core_web_sm')
    matched_sents = []  # Collect data of matched sentences to be visualized
    #filename = 'test.txt'
    filename = None
    dataset = load_text_files(filename)
    prep = Preprocess()
    Token.set_extension("grouping_type", default=None, force=True)
    Token.set_extension("list", default=None, force=True)
    place_lemmas = ['place', 'village', 'county', 'city', 'district', 'region', 'state', 'capital', 'country', 'continent', 'world']
    #print(feature_extraction('in'))
    """
    new_dataset = {}
    coref = neuralcoref.NeuralCoref(nlp.vocab)
    nlp.add_pipe(coref, name='neuralcoref', after='ner')
    for filename in dataset:
        new_dataset[filename] = []
        for paragraph in dataset[filename]:
            paragraph =  prep.remove_unwanted_character(paragraph).lstrip()
            doc = nlp(paragraph)
            new_dataset[filename].append(doc._.coref_resolved)
    nlp.remove_pipe("neuralcoref")
    """
    nlp.add_pipe(merge_ents, name='merge_ents', after='ner')
    nlp.add_pipe(merge_siblings, name='merge_siblings', after='merge_ents')
    nlp.add_pipe(merge_ancestors, name='merge_ancestors', after='merge_siblings')
    nlp.add_pipe(extract_place_in_place, name='extract_place_in_place', after='merge_ancestors')
    """
    dataset = ["In January 2015, Amazon Web Services acquired Annapurna Labs, an Israel-based microelectronics company reputedly for US$350370M.",
                "The proposed deal would give AT&T significant holdings in the media industry; AT&T's competitor Comcast had previously acquired NBCUniversal in a similar bid to increase its media holdings, in concert with its ownership of television and internet providers.",
                "BellSouth, acquired by AT&T in 2006.",
                "Berkshire acquired Russell Corporation on August 2, 2006 for $600 million or $18.00 per share.",
                "Twitch is a live streaming platform for video, primarily oriented towards video gaming content. The service was first established as a spin-off of a general-interest streaming service known as Justin.tv. Its prominence was eclipsed by that of Twitch, and Justin.tv was eventually shut down by its parent company in August 2014 in order to focus exclusively on Twitch. Later that month, Twitch was acquired by Amazon for $970 million. Through Twitch, Amazon also owns Curse, Inc., an operator of video gaming communities and a provider of VoIP services for gaming. Since the acquisition, Twitch began to sell games directly through the platform, and began offering special features for Amazon Prime subscribers."
                "Dallas (), officially the City of Dallas, is a city in the U.S. state of Texas and the seat of Dallas County, with portions extending into Collin, Denton, Kaufman and Rockwall counties. With an estimated 2017 population of 1,341,075, it is the ninth most-populous city in the U.S. and third in Texas after Houston and San Antonio. It is also the eighteenth most-populous city in North America as of 2015. Located in North Texas, the city of Dallas is the main core of the largest metropolitan area in the Southern United States and the largest inland metropolitan area in the U.S. that lacks any navigable link to the sea. It is the most populous city in the Dallas–Fort Worth metroplex, the fourth-largest metropolitan area in the country at 7.3 million people as of 2017. The city's combined statistical area is the seventh-largest in the U.S. as of 2017, with 7,846,293 residents."]
     """
    #Praveen
    from spacy.pipeline import EntityRuler
    definition,synonyms,antonyms,hyper,hypo,mero,holo = feature_extraction('acquired')            
    ruler = EntityRuler(nlp)
    buy_tags = ["acquire","purchase","buy","brought"]
    patterns = [{"label": "BUY", "pattern": "purchase"},{"label": "BUY", "pattern": "acquisition"},{"label": "BUY", "pattern": "buy"},{"label": "BUY", "pattern": "acquire"},{"label": "BUY", "pattern": "purchased"},{"label": "BUY", "pattern": "purchased by"},{"label": "BUY", "pattern": "acquired by"},{"label": "BUY", "pattern": "brought"},{"label": "BUY", "pattern": "acquired"}]
    ruler.add_patterns(patterns)
    nlp.add_pipe(ruler, name='ruler', after='extract_place_in_place')
    #Praveen
    for filename in dataset:
        result  = {"document":filename, "extraction":[]}
        for paragraph in dataset[filename]:
            print("----------------------------PRAGRAPH TOKENIZATION----------------------------------------------")
            print("current-paragraph:",paragraph)
            paragraph = prep.sentence_tokenize(paragraph)
            print("----------------------------SENTENCE TOKENIZATION----------------------------------------------")
            print("current-sentences:",paragraph)
            for each_sent in paragraph:
                print("----------------------------LEMMATIZATION--------------------------------------------------")
                lem_sent = prep.lemmatize(each_sent)
                print("lemmatized_sentence:",lem_sent)
                print("----------------------------STOP-WORD REMOVAL----------------------------------------------")
                cleaned_sentence = prep.remove_stopwords(lem_sent)
                print('stop word removed sentence:',cleaned_sentence)
                print("----------------------------WORD TOKENIZATION----------------------------------------------")
                cleaned_sentence = prep.word_tokenize(cleaned_sentence)
                print('word tokenized sentence:',cleaned_sentence)
                print("---------------------------POS TAGGED SENTENCE---------------------------------------------")
                cleaned_sentence = prep.pos_tag(cleaned_sentence)
                print('pos tagged sentence:',cleaned_sentence)
              
                each_sent = each_sent.replace('US','US$')
                sentence = nlp(each_sent)
                
                definition,synonyms,antonyms,hypernym,hyponym,meronym,holonym = feature_extraction("Purchase")
                print("word definition",definition)
                print("synonyms",synonyms)
                print("antonyms",antonyms)
                print("hypernym",hypernym)
                print("hyponym",hyponym)
                print("meronym",meronym)
                print("holonym",holonym)
                
                
                print("------------------------------NAMED ENTITY RECOGNITION---------------------------------------")
                displacy.render(sentence, style="ent",jupyter=True)
                print("------------------------------DEPENDENCY PARSING---------------------------------------------")
                displacy.render(sentence, style="dep",jupyter=True)
                print('********************************************************************************************')
                print()
                
                #Praveen
                for tag in buy_tags:
                    if tag in each_sent:
                        sentence = merge_ents(sentence)
                        print(extract_all_named_entities(sentence))
                        #for ent in sentence.ents:
                            #print(ent.text,"-->", ent.label_)
                        displacy.render(sentence, style="ent",jupyter=True)
                        displacy.render(sentence, style="dep",jupyter=True)
                #Praveen
               
                #arr = [token for token in doc if token.ent_type_=='GPE']
                #if (len(arr)>1):
                #    print(doc)
                #print_token_attr(doc)
        #displacy.render(matched_sents, style="ent", manual=True)
        json_object = json.dumps(result, indent = 4)
        print(json_object)
        with open(filename[:-4]+".json", "w") as outfile:
            outfile.write(json_object)
        #exit()
    nlp.remove_pipe("merge_ents")
    nlp.remove_pipe("merge_siblings")
    nlp.remove_pipe("merge_ancestors")
    nlp.remove_pipe("extract_place_in_place")
    nlp.remove_pipe("ruler")
    print(time.time()-s)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WINDOWS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WINDOWS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WINDOWS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WINDOWS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


----------------------------PRAGRAPH TOKENIZATION----------------------------------------------
current-paragraph: Amazon.com, Inc. (), is an American multinational technology company based in Seattle that focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence. It is considered one of the Big Four tech companies, along with Google, Apple, and Facebook. It's been referred to as "one of the most influential economic and cultural forces in the world."
----------------------------SENTENCE TOKENIZATION----------------------------------------------
current-sentences: ['Amazon.com, Inc. (), is an American multinational technology company based in Seattle that focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence.', 'It is considered one of the Big Four tech companies, along with Google, Apple, and Facebook.', 'It\'s been referred to as "one of the most influential economic and cultural forces in the world."']
-------------------

------------------------------DEPENDENCY PARSING---------------------------------------------


********************************************************************************************

----------------------------LEMMATIZATION--------------------------------------------------
lemmatized_sentence: It is considered one of the Big Four tech company , along with Google , Apple , and Facebook .
----------------------------STOP-WORD REMOVAL----------------------------------------------
stop word removed sentence: It considered one Big Four tech company , along Google , Apple , Facebook .
----------------------------WORD TOKENIZATION----------------------------------------------
word tokenized sentence: ['It', 'considered', 'one', 'Big', 'Four', 'tech', 'company', ',', 'along', 'Google', ',', 'Apple', ',', 'Facebook', '.']
---------------------------POS TAGGED SENTENCE---------------------------------------------
pos tagged sentence: [('It', 'PRP'), ('considered', 'VBD'), ('one', 'CD'), ('Big', 'NNP'), ('Four', 'CD'), ('tech', 'NN'), ('company', 'NN'), (',', ','), ('along', 'IN'), 

------------------------------DEPENDENCY PARSING---------------------------------------------


...

------------------------------DEPENDENCY PARSING---------------------------------------------


********************************************************************************************

----------------------------PRAGRAPH TOKENIZATION----------------------------------------------
current-paragraph: Statistically improbable phrases â Amazon.com's phrase extraction technique for indexing books
----------------------------SENTENCE TOKENIZATION----------------------------------------------
current-sentences: ["Statistically improbable phrases â\x80\x93 Amazon.com's phrase extraction technique for indexing books"]
----------------------------LEMMATIZATION--------------------------------------------------
lemmatized_sentence: Statistically improbable phrase â Amazon.com 's phrase extraction technique for indexing book
----------------------------STOP-WORD REMOVAL----------------------------------------------
stop word removed sentence: Statistically improbable phrase â Amazon.com 's phrase extraction technique indexing book
----------------------------WORD TOKENIZATION------

------------------------------DEPENDENCY PARSING---------------------------------------------


********************************************************************************************

----------------------------PRAGRAPH TOKENIZATION----------------------------------------------
current-paragraph: 
----------------------------SENTENCE TOKENIZATION----------------------------------------------
current-sentences: []


TypeError: Object of type Token is not JSON serializable